## Mammals

Dataset used here:
1) Terrestrial mammal area of habitat (IUCN) available at https://www.iucnredlist.org/resources/spatial-data-download (1 July 2025)
3) Bird Species' Area of Habitat (AOH) maps  requested from https://datazone.birdlife.org/contact-us/request-our-data (1 July 2025)
4) All species' habitat, minimum and maximum altitude are collected through IUCN Redlist Webpage by entering species' name one by one (July 2025)
5) Bird' IUCN redlist category available at https://datazone.birdlife.org/about-our-science/taxonomy (25 July 2025)
6) Spatial Database of Planted Forest (WRI) downloaded from https://www.wri.org/research/spatial-database-planted-trees-sdpt-version-2 (July 2025)
7) Borneo Boundary (gadm41_Borneo_IDN_1) downloaded at https://gadm.org/ simplify vertex at 200m using ArcgisPro, explode and delete small-scattered islands except :
   - Maya Karimata island -1.1114594, 109.5900927 & -0.807859, 109.441845
   - Laut Selatan island -3.67704706, 116.1493258
   - Sebuku island -3.515927, 116.3694682
   - Mahakam Delta -0.663428, 117.401863
   - and Bulungan River Estuary 3.027885, 117.468919

In [7]:
import numpy as np
import matplotlib as plt
import geopandas as gpd
import os

In [3]:
os.getcwd()

'C:\\Users\\nooriza maharani\\Documents\\Dissertation'

In [4]:
mammal = gpd.read_file('MAMMALS_TERRESTRIAL_ONLY.shp')
borneo_boundary = gpd.read_file('gadm41_Borneo.shp')

In [5]:
bird = gpd.read_file('BOTW_2024_2.gpkg')

C:\Anaconda\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'BOTW_2024_2.gpkg': 'all_species' (default), 'main_BL_HBW_Checklist_V9'. Specify layer parameter to avoid this warning.
  result = read_func(


In [6]:
import fiona

# List all layers in the GPKG file
layers = fiona.listlayers("BOTW_2024_2.gpkg")
print(layers)

['all_species', 'main_BL_HBW_Checklist_V9']


In [7]:
borneo_boundary.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
mammal.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
bird.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Clipping Mammals with Borneo Boundary

In [11]:
# Cek geometri yang tidak valid
invalid = mammal[~mammal.is_valid]
print(invalid)

      id_no             sci_name  presence  origin  seasonal      compiler  \
2535   9760  Helarctos malayanus         5       1         1  Graham Usher   
9831  41772         Sus barbatus         1       1         1          IUCN   

      yrcompiled                                           citation  \
2535        2017               Scotson, L. and Fredriksson, G. 2016   
9831        2017  IUCN (International Union for Conservation of ...   

     subspecies subpop  ... marine terrestria freshwater  SHAPE_Leng  \
2535       None   None  ...  false       true      false  298.047663   
9831   barbatus   None  ...  false       true      false  203.352979   

      SHAPE_Area           area  \
2535   28.227425  347094.755252   
9831   29.450292  362205.059335   

                                                habitat altitude max_alt  \
2535          Forest, Shrubland, Artificial/Terrestrial      1.0  3000.0   
9831  Forest, Wetlands (inland), Marine Neritic, Mar...      0.0     0.0   


In [12]:
# Select animals which presence is not extinct and resident through the year
mammals_filtered = mammal[(mammal['presence'] != 5) & (mammal['seasonal'] == 1)].copy()

In [13]:
len(mammals_filtered)

12450

In [ ]:
from shapely.validation import make_valid

mammals_filtered["geometry"] = mammals_filtered["geometry"].apply(make_valid)
borneo_boundary["geometry"] = borneo_boundary["geometry"].apply(make_valid)

In [ ]:
mammals_borneo = gpd.clip(mammals_filtered, borneo_boundary)

In [ ]:
# Plot the clipped layer
mammals_borneo.plot(figsize=(10, 10), edgecolor='black', cmap='Set2')
plt.title("Clipped Mammals Layer")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
# Select animals where yrcompiled > 2000 and category as CR, VU, EN
mammals_threatened = mammals_borneo[
    (mammals_borneo['category'].isin(['VU', 'CR', 'EN'])) &
    (mammals_borneo['yrcompiled'] > 2000)
].copy()

In [ ]:
len(mammals_threatened)

In [ ]:
# Reproject to Southeast Asia Albers Equal Area (ESRI:102025) in square metres
mammals_borneo_proj = mammals_threatened.to_crs("ESRI:102025")

# Calculate area in square meters
mammals_borneo_proj['area_m2'] = mammals_borneo_proj.geometry.area
mammals_borneo_proj['area_km2'] = mammals_borneo_proj['area_m2'] / 1_000_000
# Remove small areas possibly the result of clipping process
mammals_borneo_proj = mammals_borneo_proj[mammals_borneo_proj['area_km2'] >= 80]

In [ ]:
print(mammals_borneo_proj.columns.to_list())

In [ ]:
# Sort by SHAPE_Area in descending order
mammals_sorted = mammals_borneo_proj.sort_values(by='area_km2', ascending=False)

# Display selected columns
#print(mammals_sorted[['sci_name', 'category', 'area_km2']])

In [ ]:
# Merged duplicate species name
# Sort to keep largest area per species
sorted_gdf = mammals_borneo_proj.sort_values('area_km2', ascending=False)

# Group and pick first (largest area)
mammals_merged = (
    sorted_gdf
    .groupby('sci_name', as_index=False)
    .agg({
        'category': lambda x: ', '.join(sorted(set(x))),
        'yrcompiled': lambda x: ', '.join(map(str, sorted(set(x)))),
        'area_km2': 'first',
        'geometry': 'first'
    })
)

# Convert to GeoDataFrame again
import geopandas as gpd
mammals_merged = gpd.GeoDataFrame(mammals_merged, geometry='geometry', crs=mammals_borneo_proj.crs)

In [ ]:
# Sort by SHAPE_Area in descending order
mammals_sort = mammals_merged.sort_values(by='sci_name', ascending=False)

# Display selected columns
#print(mammals_sort[['sci_name', 'category', 'yrcompiled', 'area_km2']])

In [ ]:
len(mammals_merged)

In [ ]:
iucn_data = pd.read_excel('mammals_database.xls')

In [ ]:
mammal_aoh = mammals_merged.merge(iucn_data, on = 'sci_name', how = 'left')

In [ ]:
mammal_aoh= mammal_aoh.dropna(subset=['max_alt'])

In [ ]:
len(mammal_aoh)

In [ ]:
#mammals_merged.to_file("mammals_merged.gpkg", driver = "GPKG")

In [ ]:
mammal_aoh.to_file("mammal_aoh.shp")

### Clipping Birds with Borneo Boundary

In [ ]:
# Invalid Geometry
invalid_bird = bird[~bird.is_valid]
print(invalid1)

In [ ]:
bird_borneo = gpd.clip(bird, borneo_boundary)

In [ ]:
len(bird_borneo)

In [ ]:
# Plot the clipped layer
bird_borneo.plot(figsize=(10, 10), edgecolor='black', cmap='Set2')
plt.title("Clipped bird Layer")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
# Select animals which presence is not extinct and resident through the year
bird_borneo_f = bird_borneo[(bird_borneo['presence'] != 5) & (bird_borneo['seasonal'] == 1) & (bird_borneo['yrcompiled'] > 2000)].copy()

In [ ]:
iucn_data_bird = pd.read_excel('birds_iucn_taxonomy.xls')

In [ ]:
bird_aoh = bird_borneo_f.merge(iucn_data_bird, on = 'sci_name', how = 'left')

In [ ]:
birds_threatened_aoh = bird_aoh[bird_aoh['iucn_category'].isin(['VU', 'CR', 'EN'])].copy()

In [ ]:
len(birds_threatened_aoh)

In [ ]:
# Reproject to Southeast Asia Albers Equal Area (ESRI:102025) in square metres
bird_proj = birds_threatened_aoh.to_crs("ESRI:102025")

# Calculate area in square meters
bird_proj['area_m2'] = bird_proj.geometry.area
bird_proj['area_km2'] = bird_proj['area_m2'] / 1_000_000

In [ ]:
# Merged duplicate species name
# Sort to keep largest area per species
bird_gdf = bird_proj.sort_values('area_km2', ascending=False)

# Group and pick first (largest area)
bird_merged = (
    bird_gdf
    .groupby('sci_name', as_index=False)
    .agg({
        'iucn_category': lambda x: ', '.join(sorted(set(x))),
        'yrcompiled': lambda x: ', '.join(map(str, sorted(set(x)))),
        'area_km2': 'first',
        'geometry': 'first'
    })
)

# Convert to GeoDataFrame again
import geopandas as gpd
bird_merged = gpd.GeoDataFrame(bird_merged, geometry='geometry', crs=bird_proj.crs)

In [ ]:
# Sort by SHAPE_Area in descending order
bird_sort = bird_merged.sort_values(by='area_km2', ascending=False)

# Display selected columns
print(bird_sort[['sci_name', 'iucn_category', 'yrcompiled', 'area_km2']])

In [ ]:
altitude = pd.read_excel('birds_database.xls')

In [ ]:
aoh_bird = bird_merged.merge(altitude, on = 'sci_name', how = 'left')

In [ ]:
print(aoh_bird)

In [ ]:
aoh_bird= aoh_bird.dropna(subset=['max_alt'])
len(aoh_bird)

In [ ]:
aoh_bird.to_file("aoh_bird.shp")

## Selecting Planted Forest 

In [19]:
conda install -c conda-forge geopandas fiona gdal

^C

Note: you may need to restart the kernel to use updated packages.
Retrieving notices: done
Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Anaconda

  added / updated specs:
    - fiona
    - gdal
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.6               |       h4190f5b_0         100 KB
    ca-certificates-2025.7.14  |       h4c7d964_0         152 KB  conda-forge
    certifi-2025.7.14          |     pyhd8ed1ab_0         156 KB  conda-forge
    click-plugins-1.1.1.2      |     pyhd8ed1ab_0          12 KB  conda-forge
    curl-8.14.1                |       heda7e69_0         191 KB
    expat-2.7.1                |       hac47afa_0         229 KB  conda-forge
    fiona-1.10.1               |  py312h885b0b7_0         1.0 MB
    folium-0.20.0              |    

In [13]:
print(os.getcwd())

C:\Users\nooriza maharani\Documents\Dissertation


In [17]:
import geopandas as gpd
import fiona
# Path to your .gdb
gdb_path = r"C:\Users\nooriza maharani\Documents\Dissertation\sdpt_v21_v09152024_public.gdb"

# List layers in the GDB
layers = fiona.listlayers(gdb_path)
print("Layers:", layers)

# Load a specific layer
gdf = gpd.read_file(gdb_path, layer=layers[0])  # Or specify by name
print(gdf.head())

DriverError: Failed to open dataset (flags=68): C:/Users/nooriza maharani/Documents/Dissertation/sdpt_v21_v09152024_public.gdb